In [ ]:
# a graph with n vertices, start from s, end to v
# If we assume the optimal shortest distance s->v solution for num_of_edges <= i (0-n) is P(v)
# and P'(v) is the optimal solution for <=i-1
# then if edges of P < i, P == P'
# elif edges of P(v) = i, the last vertex is v, w is the vertex before reaching to v
# P(w) must be the optimal solution for s->w for edges<=i-1

# Unlike Dijkstra, Bellman Ford also handles negative values
# O(mn^2) relevant to indegree of vertice


### If assume no negative cycle, can stop early when for every v, d(i,v) = d(i-1,v)
<br>as for every v, d(i,v) = d(i-1,v)
<br>so d(i,v) = d(i-1,v); d(i,w) = d(i-1,w)
<br>so d(i-1,v) <= d(i-1,w) + d(wv)
<br>as d(i+1,v) = min(d(i,v), d(i,w)+d(wv))
<br>which is also min(d(i-1,v), d(i-1,w)+d(wv))
<br>so d(i,v) must <= d(i,w)+d(wv)
<br>so d(i+1,v) = d(i,v) = d(i-1,v)
<br>依此类推，最短距路不会再improve,全都是d(i-1,v)

### If and only if for every v in graph with n nodes, if d(n, v) = d(n-1,v) then graph has no negative cycle
<br>1st => proof: If there is a d(n,v) != d(n-1,v) then must have negative cycle
<br>as can not pass a node twice, d(n,v) here has n edges, there must be a cycle, as a positive cycle can be removed, then must be a negative cycle.
<br>2nd <= proof: If for every node v, d(n,v) = d(n-1,v), then no negative cycle
<br>https://algo.epfl.ch/_media/en/courses/2011-2012/algorithmique-cycles-2011a.pdf
<br>"bellman_ford_negative_cycle_proof.pdf"

In [2]:
import re
from collections import defaultdict

def file_to_graph(filename):  # head
    with open(filename) as f:
        content = f.readlines()    
    pattern = re.compile('\d+')
    matches = [re.findall(pattern, e) for e in content]  
    # first line is number of nodes and edges
    num_of_nodes, num_of_edges = [int(x) for x in matches[0]]
    integers = [[int(x) for x in n] for n in matches[1:]]
    graph = defaultdict(list)
    for x in integers:
        graph[x[0]].append({'head': x[1], 'length': x[2]})
    return graph

def file_to_graph_(filename):  # tail
    with open(filename) as f:
        content = f.readlines()    
    pattern = re.compile('\d+')
    matches = [re.findall(pattern, e) for e in content]  
    # first line is number of nodes and edges
    num_of_nodes, num_of_edges = [int(x) for x in matches[0]]
    integers = [[int(x) for x in n] for n in matches[1:]]
    graph = defaultdict(list)
    for x in integers:
        graph[x[1]].append({'tail': x[0], 'length': x[2]})
    return graph

In [3]:
graph = {
    'A': [],
    'B': [{'tail': 'A', 'length': 1}],
    'C': [{'tail': 'A', 'length': 3}, {'tail': 'B', 'length': 1}],
    'D': [{'tail': 'C', 'length': 1}, {'tail': 'B', 'length': 5}],
    'E': [{'tail': 'C', 'length': 2}, {'tail': 'D', 'length': 2}]
}

In [4]:
import numpy as np
import pandas as pd

def bellman(graph, s):  # s is source
    nodes = list(graph.keys())
    n = len(nodes)
    df = pd.DataFrame(np.full((n+1, n), np.inf), columns=nodes)
    df.loc[:, s] = 0
    nodes.remove(s)
    for i in range(1, n+1):
        for v in nodes:
            pre_nodes = [x['tail'] for x in graph[v]]
            pre_length = [x['length'] for x in graph[v]]
            zip_ = list(zip(pre_nodes, pre_length))
            min_pre = np.inf
            if zip_:
                for n in zip_:
                    if df.loc[i-1, n[0]] + n[1]< min_pre:
                        min_pre = df.loc[i-1, n[0]] + n[1]
            df.loc[i, v] = min(df.loc[i-1, v], min_pre)
    
    assert list(df.iloc[-1, :].values) == list(df.iloc[-2, :].values)  # no negative cycle
    return df.iloc[-1, :]  # shortest distance from source to each node


In [5]:
bellman(graph, 'A')


A    0.0
B    1.0
C    2.0
D    3.0
E    4.0
Name: 5, dtype: float64